# 日度单品利润分析报告

---

*date: 2021/7/6*

*Author: wali*

In [3]:
'''
所需函数库
'''
import pandas as pd
import numpy as np
from datetime import datetime
import re
import os
import json

In [4]:
# 6月份数据

# #数据存放根目录：存放在程序所在文件夹的同目录

# dir = '..\\data\\'

# resultDir = '..\\data\\result\\'


# # 生意经数据
# syjFileAddress = dir + 'syj\\order-20210600.csv'
# # 商品基本信息
# productInfoFileAddress = dir + 'productInfo\\productInfo.xlsx'
# # 聚水潭AB单订单
# jst_ABorderAddress = dir + 'jst\\销售主题分析_订单(明细商品)_20210601_20210630.xlsx'
# # 聚水潭客服退差
# jst_diffAddress = dir + 'jst\\diffmoney_订单(明细商品)_20210601_20210630.xlsx'
# # 微信AB单退差
# wxBDAddress = dir + 'wxBD\\微信退差汇总6月份.xlsx'

# # 生意参谋GMV
# cycmFilesAddress = dir + 'sycm\\sycmallfiles\\'

# #直通车
# ztcFileAddress = dir + 'ztc\\财务部直通车数据210600.csv'
# # 淘宝客
# tbkFilesAddress = dir + 'tbk\\tbkFiles\\'
# index_IDAddress = dir + 'tbk\\淘客单品关联_2021-06-01_2021-06-30.xlsx'
# # 超级推荐
# cjtjFileAddress = dir + 'cjtj\\商品推广_创意分日报表_20210600.xls'
# # 运费
# yfFileAddress = dir + 'yf\\yf.xlsx'
# # AI投放
# aitfFileAddress = dir + 'aitf\\AI投放_210600.xls'


In [3]:
# 7月份数据

#数据存放根目录：存放在程序所在文件夹的同目录

dir = '..\\data\\'

resultDir = '..\\data\\result\\'


# 生意经数据
syjFileAddress = dir + 'syj\\order-20210700.csv'
# 商品基本信息

product1FileAddress = dir + '\\productInfo\\商品信息.xlsx'
product2FileAddress = dir + '\\productInfo\\组合商品.xlsx'

# 聚水潭AB单订单
jst_ABorderAddress = dir + 'jst\\销售主题分析_订单(明细商品)_20210701_20210731.xlsx'
# 聚水潭客服退差
jst_diffAddress = dir + 'jst\\diffmoney_订单(明细商品)_20210701_20210731.xlsx'
# 微信AB单退差

wxBD_li = ['7.1','7.2','7.3','7.4','7.5','7.6','7.7']
wxBDAddress = dir +'wxBD\\7.7 旗舰店微信销售发一份.xlsx'


# 生意参谋GMV
cycmFilesAddress = dir + 'sycm\\sycmallfiles\\'

#直通车
ztcFileAddress = dir + 'ztc\\财务部直通车数据210700.csv'
# 淘宝客
tbkFilesAddress = dir + 'tbk\\tbkFiles\\21年7月\\'
index_IDAddress = dir + 'tbk\\淘客单品关联_2021-07-01_2021-07-07.xlsx'
# 超级推荐
cjtjFileAddress = dir + 'cjtj\\商品推广_创意分日报表_20210700.xls'
# 运费
yfFileAddress = dir + 'yf\\yf.xlsx'
# AI投放
aitfFileAddress = dir + 'aitf\\AI投放_210700.xls'


### 1.生意经数据下载

In [4]:
syjData = pd.read_csv(syjFileAddress,encoding='gbk')

# 把生意经中【商家编码】空格去除，B8809,B7053全部置换位空
syjData['商家编码'] = syjData['商家编码'].str.replace(" ", "").str.replace("B8809","").str.replace("B7053","").str.upper()
# 去除“=……”
syjData['订单号']=syjData['订单号'].str[2:-1]
# 付款时间字符串str格式转化为日期格式
syjData['付款时间'] = pd.to_datetime(syjData['付款时间'])
# 再统一为字符串'%Y%m%d'格式
syjData['付款时间'] = syjData['付款时间'].dt.strftime('%Y%m%d')
# 以宝贝链接的后12位数字作为商品ID，将商品ID插入第一列
syjData.insert(0,'商品ID',syjData['宝贝链接'].str[-12:])


In [5]:
syjData.iloc[1,:]

syjData.groupby(['付款时间']).sum()

,子订单金额,成交单价,宝贝数量,子订单金额（折算:邮费、优惠等）
付款时间,,,,
20210701,154547.87,153660.67,809,146217.01
20210702,149719.15,149030.88,751,142631.37
20210703,123973.92,123669.92,578,120900.97
20210704,138473.05,137627.71,690,134305.00
20210705,144326.45,143400.60,722,139961.32
20210706,156875.90,155099.36,812,150143.13
20210707,178575.80,161373.55,884,171532.10
20210708,130154.66,130126.66,632,123916.90


### 2. 将运营成本（单品的运营成本）增加至主表
先根据【商品基本资料】把其它价格1 添加至 主表
下载聚水潭-销售主题分析-筛选标记多标签数据和小红旗订单-下载
把聚水潭的订单中[【标记多标签】为'AB单'或'特殊单'，并且商家编码！=商品编码的订单号 ] or [【子订单金额（折算:邮费、优惠等）】=0 AND 【宝贝数量】>0 AND 商家编码 <> 商品编码]，把生意经里的该订单号商品的其它价格1更换为聚水潭的其它价格1，运营成本=其它价格1 * 销售数量（因为是AB单，所以真实运营成本应该取聚水潭的真实发货数据）
WHERE (标记多标签 like '%特殊单%' or 标记多标签 like '%AB单%')

AND 价格为零的商品数量 is null
AND 线上商品名 not like '%无须发货%' 
AND 线上商品名 is not null 
AND 线上商品名 not like '%纸袋%' 

In [6]:
products = []
products.append(pd.read_excel(product1FileAddress)[['商品编码','商品名','其它价格1']])
p2 = pd.read_excel(product2FileAddress)[['组合商品编码','组合商品名称','其它价格1']]
p2.columns=['商品编码','商品名','其它价格1']
products .append(p2)
productInfoData = pd.concat(products)
productInfoData.columns = ['商家编码','商品名','其它价格1']

productInfoData.drop_duplicates(subset='商家编码', keep='first', inplace=True)
productInfoData.to_excel(dir + 'productInfo\\商品编码-名称-其它价格1.xlsx',index=False)


# productInfoData =  pd.read_excel(productInfoFileAddress)
# 将商家编码统一转化为字符串格式
productInfoData['商家编码'] = productInfoData['商家编码'].apply(str) 
# 以商家编码作为链接，拼接商品基本信息和生意经数据
syjData_cost = pd.merge(syjData,productInfoData,on='商家编码',how ="left")


'''
由于购物金,生意经中商家编码为空、商品基本资料里邮费成本为空，
再加上 2个商品编码 商品资料维护问题，共100余个空值
'''
# 没有运营成本的商品有多少个？
print('订单中没有运营成本的商品编码有{}个,共{}个订单'.format(len(   syjData_cost[syjData_cost['其它价格1'].isna()].drop_duplicates(subset='商家编码', keep='first', inplace=False)  ),len(   syjData_cost[syjData_cost['其它价格1'].isna()])))
print('\n商家编码和名称分别是(这些没有运营价格的订单已经保存至结果目录文件中):\n\n',syjData_cost[syjData_cost['其它价格1'].isna()].drop_duplicates(subset='商家编码', keep='first', inplace=False)[['商家编码','商品名']])
syjData_cost[syjData_cost['其它价格1'].isna()].to_excel(resultDir + '订单中没有运营成本的订单列表.xlsx',index=False)

# 去除AB单之前的总运营成本
beforeCost = (syjData_cost['其它价格1']* syjData_cost['宝贝数量']).sum()

c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


订单中没有运营成本的商品编码有4个,共40个订单

商家编码和名称分别是(这些没有运营价格的订单已经保存至结果目录文件中):

               商家编码  商品名
590  6970839519490  NaN
648            NaN  NaN
711  6970839519506  NaN
856       ZP000008   邮费


In [ ]:
## TEST

syjData_cost

### 3.下载聚水潭-销售主题分析数据-AB单订单（商品）明细

注意设置 标记多标签为AB单或特殊单,取消已发货\组合商品拆分

In [7]:
jst_ABorder = pd.read_excel(jst_ABorderAddress)

syjData_costAB = syjData_cost
syjData_costAB.insert(syjData_costAB.shape[1],'是否AB单',0)


# 去除价格为零的商品数据和线上商品名为无需发货
jst_ABorder = jst_ABorder[(jst_ABorder['价格为零的商品数量'].isna())&(jst_ABorder['线上商品名'].str.contains('无须发货')==False)]
print('去重订单号之后的jst中AB单的订单数量 {}'.format(len(jst_ABorder.drop_duplicates(subset='线上订单号',keep='first',inplace=False))))



# ###############################################################################################




syj_index = syjData_costAB[syjData_costAB['订单号'].isin(jst_ABorder['线上订单号'])].index


# ###############################################################################################

syjData_costAB['是否AB单'][syj_index]=1
print('生意经中有{}个订单号在聚水潭AB单里'.format(len(syj_index)))

temp = 0 
# 将生意经中AB单的订单的运营成本替换
print('这些AB单的订单号一致，聚水潭商品编码==生意经中的商家编码, 即，真实发了这些产品：')
for i in syj_index:
    jst_index = jst_ABorder[jst_ABorder['线上订单号']==syjData_costAB['订单号'][i]].index
    for j in jst_index:
        
        if syjData_costAB['商家编码'][i] != jst_ABorder['商品编码'][j]:
            if '赠品' in syjData_costAB['宝贝标题'][i]:
                syjData_costAB['其它价格1'][i]=0
            
            else:
                syjData_costAB['其它价格1'][i]=jst_ABorder['其它价格1'][j]
                
            temp = temp + 1
                
        else:
            print(syjData_costAB['订单号'][i]," ",syjData_costAB['商家编码'][i]," ",jst_ABorder['商品编码'][j]," ",syjData_costAB['宝贝标题'][i])
            

print('生意经中有{}个订单的运营价格被替换'.format(temp))


# 去除AB单之后的总运营成本
afterCost = (syjData_costAB['其它价格1']* syjData_costAB['宝贝数量']).sum()
# 订单（去退）支付总金额
payMoney = (syjData_costAB['子订单金额（折算:邮费、优惠等）']* syjData_costAB['宝贝数量']).sum()

print('\n\n')

print('去除AB单之前的总运营成本',beforeCost)
print('去除AB单之后的总运营成本',afterCost)
print('预计可去除AB单成本:',beforeCost-afterCost)

# AB相关信息统计分析数据保存
syjData_costAB.to_excel(resultDir + 'syjData_costAB.xlsx',index=False)

c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


去重订单号之后的jst中AB单的订单数量 631
生意经中有643个订单号在聚水潭AB单里
这些AB单的订单号一致，聚水潭商品编码==生意经中的商家编码, 即，真实发了这些产品：


c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

1925578225544511020   TM000202009050005   TM000202009050005   贝易早教机儿童玩具婴儿智能讲故事机器人儿歌播放器音乐启蒙益智
1925934734113727239   6970839515515   6970839515515   贝易手摇铃新生婴儿玩具0-1岁牙胶可咬宝宝3个月礼物礼盒抓握益智
1925934734113727239   ZP000027   ZP000027   贝易儿童成长卡【赠品单拍不发货】
生意经中有643个订单的运营价格被替换



去除AB单之前的总运营成本 659892.5
去除AB单之后的总运营成本 569814.7
预计可去除AB单成本: 90077.80000000005


In [8]:
len(jst_ABorder)

742

### 3. 将退差数据添加至主表

下载聚水潭销售主题分析-【beie店铺】的所有备注退差数据

In [9]:
jst_diffOrder = pd.read_excel(jst_diffAddress,engine="openpyxl")
# 筛选出卖家备注不为空并且含有￥的行
jst_diffOrderMoney = jst_diffOrder[pd.notnull(jst_diffOrder['卖家备注'])&(jst_diffOrder['卖家备注'].str.contains('￥'))][['内部订单号','线上订单号','卖家备注','买家账号','订单日期','商品编码','售价','其它价格1']]


'''
1. 客服退差数据清洗
划重点，这里对聚水潭的AB单进来了进行了简单排序
排序规则：内部订单号升序，其它价格1降序，即价格高的商品排在第一个
去重规则，keep='first',保留第一个重复行

'''
jst_diffOrderMoney_sort_drop = jst_diffOrderMoney.sort_values(by=['内部订单号', '其它价格1'], ascending=[True, False]).drop_duplicates(subset='卖家备注', keep='first', inplace=False)


reg = r"￥(((\d)|(b\d))(.*)链接|(链接换购(.*)链接))*￥"

jst_customerDiffMoney = jst_diffOrderMoney_sort_drop
jst_customerDiffMoney.insert(jst_customerDiffMoney.shape[1],'备注金额',0)



# 拿到里面的金额
li = list()
for i in range(len(jst_customerDiffMoney)):
    if re.search(reg,jst_customerDiffMoney["卖家备注"].iloc[i]) is None:
#         print("\n 错误,第{}行数据, 待更改的卖家备注：{}\n".format(i,jst_customerDiffMoney["卖家备注"][i]))
        wmoney =  re.findall(r"\d+\.?\d*",jst_customerDiffMoney["卖家备注"].iloc[i])
#         if len(wmoney)==1 and float(wmoney[0])<300:
        if float(wmoney[0])<1000:
            jst_customerDiffMoney["备注金额"].iloc[i]=float(wmoney[0])
        else:
#             print('1',jst_customerDiffMoney['卖家备注'].iloc[i])
            li.append(i)
#         data_new_dropdup["卖家备注"].loc[i] = input("\n请输入更改后的卖家备注: ")
        # condition2_re()
    else:
#         jst_customerDiffMoney["卖家备注"].iloc[i] = re.search(reg,jst_customerDiffMoney["卖家备注"][i])[0]        
#         money = re.search(reg,jst_customerDiffMoney["卖家备注"].iloc[i])
        money =  re.findall(r"\d+\.?\d*",jst_customerDiffMoney["卖家备注"].iloc[i])
        try:
#             if float(money[0])<jst_customerDiffMoney["其它价格1"][i]*10:
            if float(money[0])<1000:
                jst_customerDiffMoney["备注金额"].iloc[i] = float(money[0])
            else:
#                 print('2',jst_customerDiffMoney['卖家备注'].iloc[i])
                li.append(i)
                
        except:
#             print('3',jst_customerDiffMoney['卖家备注'].iloc[i])
            li.append(i)
            
            
# 生成错误备注表格，保存至本地

jst_customerDiffMoney.to_excel(resultDir+'客服登记退差【卖家备注】排序去重订单号之后的提取备注金额信息表.xlsx',index=False)
jst_customerDiffMoney.iloc[li,:].to_excel(resultDir + '严重错误备注列表.xlsx',index=False)
# 统计备注错误信息基本情况
print('客服备注含有￥的订单数：{}，错误个数：{}，备注里提取出金额订单个数：{}，备注提取率：{:.2f}%'.format(len(jst_customerDiffMoney),len(li),len(jst_customerDiffMoney)-len(li),(1-len(li)/len(jst_customerDiffMoney))*100))


print('客服备注匹配出的最大金额'.format(jst_customerDiffMoney['备注金额'].max()))

print('客服备注登记总金额',jst_customerDiffMoney[jst_customerDiffMoney['备注金额']>0]['备注金额'].sum())



# 为连接syjData_costAB，使用订单号商家编码 作为连接
jst_customerDiffMoney_right = jst_customerDiffMoney[jst_customerDiffMoney['备注金额']>0]
jst_customerDiffMoney_right.insert(0,'订单号商家编码',jst_customerDiffMoney_right['线上订单号'].str[0:19]+jst_customerDiffMoney_right['商品编码'])


syjData_costAB_ = syjData_costAB
syjData_costAB_.insert(0,'订单号商家编码',syjData_costAB['订单号']+syjData_costAB['商家编码'])
syjData_costAB_.insert(0,'日期商品ID',syjData_costAB['付款时间']+syjData_costAB['商品ID'])

syjData_costAB_diffM = pd.merge(syjData_costAB_,jst_customerDiffMoney_right.loc[:,['订单号商家编码','备注金额']],on='订单号商家编码',how ="left")


print('备注金额的订单个数',len(syjData_costAB_diffM)-syjData_costAB_diffM['备注金额'].isna().sum())
print('排除以退款订单后的匹配备注金额',syjData_costAB_diffM['备注金额'].sum())

#    syjData_costAB_diffM = pd.merge(syjData_costAB,jst_customerDiffMoney_right.loc[:,['订单号商家编码','备注金额']],on='订单号商家编码',how ="left")


syjData_costAB_diffM.fillna(0,inplace=True)

syjData_costAB_diffM.insert(syjData_costAB_diffM.shape[1],'运营金额',syjData_costAB_diffM['宝贝数量']*syjData_costAB_diffM['其它价格1'])
# result
syjData_costAB_diffM.to_excel(resultDir+'没有聚合ID前的订单销售情况包括退差AB单标志以及成本相关.xlsx',index=False)

c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


客服备注含有￥的订单数：380，错误个数：3，备注里提取出金额订单个数：377，备注提取率：99.21%
客服备注匹配出的最大金额
客服备注登记总金额 22265.43
备注金额的订单个数 295
排除以退款订单后的匹配备注金额 16474.43


In [10]:
costPercent = syjData_costAB_diffM[syjData_costAB_diffM['子订单金额（折算:邮费、优惠等）']>0]
costPercent.insert(costPercent.shape[1],'运营成本占比',costPercent['运营金额']/costPercent['子订单金额（折算:邮费、优惠等）'])

costPercent = costPercent[costPercent['是否AB单']==0].sort_values(by=['付款时间','运营成本占比'],ascending=[True,False])


# 去除AB单的订单和正常订单的退差之后，用户有支付金额和运营成本之间的占比关系
print('去除AB单的订单之后，运营成本占用户有支付金额（去退差）',costPercent['运营金额'].sum()/(costPercent['子订单金额（折算:邮费、优惠等）'].sum()-syjData_costAB_diffM['备注金额'].sum()))

costPercent.to_excel(resultDir + '每个支付金额大于零订单且非AB单的运营成本占比表.xlsx',index=False)


# test
print(costPercent['运营金额'].sum())

print(costPercent['子订单金额（折算:邮费、优惠等）'].sum())

去除AB单的订单之后，运营成本占用户有支付金额（去退差） 0.5833673879889769
559176.7
975007.1


### 4.生成主透视表

In [11]:
main_TSB = syjData_costAB_diffM.groupby(['日期商品ID']).sum()
main_TSB['成交单价']=main_TSB['成交单价']/main_TSB['宝贝数量']
main_TSB['其它价格1'] = main_TSB['运营金额']/main_TSB['宝贝数量']

main_TSB

,子订单金额,成交单价,宝贝数量,子订单金额（折算:邮费、优惠等）,其它价格1,是否AB单,备注金额,运营金额
日期商品ID,,,,,,,,
20210701539143464966,336.00,168.000000,2,312.28,84.050000,0,0.0,168.1
20210701552028723427,28.00,28.000000,1,28.00,10.400000,0,0.0,10.4
20210701564879845606,18605.00,265.536232,69,18505.00,114.678261,8,60.0,7912.8
20210701567738064768,18291.80,157.687931,116,15614.71,68.962069,11,636.5,7999.6
20210701569762951856,558.20,69.775000,8,563.30,37.500000,0,16.0,300.0
...,...,...,...,...,...,...,...,...
20210708644818187878,1012.00,253.000000,4,866.57,131.925000,0,0.0,527.7
20210708644911208119,17.81,17.810000,1,17.81,7.300000,0,0.0,7.3
20210708646119674804,3286.10,298.736364,11,3286.10,180.818182,0,0.0,1989.0


### 5.微信补单

In [12]:
# wxData = pd.read_excel(wxBDAddress)
wxBD_day = []
for d in wxBD_li:
    wxBD_day.append(pd.read_excel(wxBDAddress,sheet_name=d))
wxData = pd.concat(wxBD_day)
wxData.insert(wxData.shape[1],'AB单标志',0)
wxData['AB单标志'][wxData['是否AB']=='是']=1
wxData[wxData['是否AB']!='是']['AB单标志']=0
# 保存一份至本地
wxData.to_excel(dir + '\\wxBD\\微信退差汇总7月份.xlsx',index =False)


wxData.insert(wxData.shape[1],'商品ID',0)
wxData = wxData[['序列','商品ID','链接','退款总金额','AB单标志']]
wxData.columns = ['日期','商品ID','链接','退款总金额','AB单标志']


for i in range(len(wxData)):
    try:
        wxData['商品ID'].iloc[i]=re.search('\d+',wxData['链接'].iloc[i])[0]

        
#         print(re.search('\d+',wxData['链接'][i])[0])
    except:
        print('no id',wxData['商品ID'].iloc[i])
        


wxData['日期'] = wxData['日期'].dt.strftime('%Y%m%d')
wxData.insert(0,'日期商品ID',wxData['日期'].str[:] + wxData['商品ID'].str[:]) 
wxData = wxData[['日期商品ID','退款总金额','AB单标志']]

wxData.fillna(0,inplace=True)
wxData_TSB = wxData.groupby(['日期商品ID']).sum()

print('微信退款总金额{}, 微信登记AB单个数{}'.format(wxData_TSB['退款总金额'].sum(),wxData_TSB['AB单标志'].sum()))

print('微信登记AB单退款金额{}'.format(wxData[wxData['AB单标志']==1]['退款总金额'].sum()))

main_TSB_wx = pd.merge(main_TSB,wxData_TSB,left_index=True,right_index=True,how='left')

main_TSB_wx.fillna(0,inplace=True)

c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\pand

微信退款总金额222048.63, 微信登记AB单个数746
微信登记AB单退款金额218167.63


### 合并两个表中的数据函数

In [13]:
# def concatXLSX(ori_dir,to_dir,to_name):
def concatXLSX(ori_dir):
    #新建列表，存放文件名（可以忽略，但是为了做的过程能心里有数，先放上）
    filename_excel = []
    #新建列表，存放每个文件数据框（每一个excel读取后存放在数据框）
    frames = []
    for root, dirs, files in os.walk(ori_dir):
        for file in files:
            #print(os.path.join(root,file))
            filename_excel.append(os.path.join(root,file))
            df = pd.read_excel(os.path.join(root,file),header=5) #excel转换成DataFrame
            frames.append(df)#
    #打印文件名
    print('文件个数',len(filename_excel))
     #合并所有数据
    result = pd.concat(frames)
#     result.to_excel('{}{}.xlsx'.format(to_dir,to_name),index = False,encoding="utf_8_sig")
    return result

### 第六步，GMV，生意参谋数据

In [14]:
# GMV，生意参谋数据

# to_name = 'cycm7月份'
# 拼接生意参谋数据
sycmData = concatXLSX(cycmFilesAddress)

sycmData['支付金额'] = sycmData['支付金额'].str.replace(',',"")
sycmData['支付金额'] = pd.to_numeric(sycmData['支付金额'])

sycmData = sycmData[['统计日期','商品ID','支付金额']]

sycmData['统计日期'] = pd.to_datetime(sycmData['统计日期'])
sycmData['统计日期'] = sycmData['统计日期'].dt.strftime('%Y%m%d')
sycmData['商品ID'] = sycmData['商品ID'].apply(str)
sycmData.insert(0,'日期商品ID',sycmData['统计日期'] + sycmData['商品ID'])

sycmData.fillna(0,inplace=True)

print('生意参谋支付金额总金额GMV：{}'.format(sycmData['支付金额'].sum()))

sycmData_TSB = sycmData[['日期商品ID','支付金额']].groupby(['日期商品ID']).sum()
main_TSB_wx_GMV = pd.merge(main_TSB_wx,sycmData_TSB,left_index=True,right_index=True,how='left') 


main_TSB_wx_GMV.fillna(0,inplace=True)

文件个数 7
生意参谋支付金额总金额GMV：1078169.26


### 第七步, 下载直通车数据

In [15]:
ztcData = pd.read_csv(ztcFileAddress)
ztcData = ztcData[['日期','商品id','花费(分)']]
ztcData['商品id'] = ztcData['商品id'].apply(str)
ztcData['日期'] = pd.to_datetime(ztcData['日期'])
ztcData['日期'] = ztcData['日期'].dt.strftime('%Y%m%d')
ztcData['花费(分)'] = ztcData['花费(分)']/100
ztcData.insert(0,'日期商品id',ztcData['日期'] + ztcData['商品id'])


ztcData.fillna(0,inplace=True)
ztc_TSB = ztcData.groupby('日期商品id').sum()
main_TSB_wx_GMV_ztc = pd.merge(main_TSB_wx_GMV, ztc_TSB, left_index=True,right_index=True,how = 'left')

print('直通车花费：',main_TSB_wx_GMV_ztc['花费(分)'].sum())

main_TSB_wx_GMV_ztc.fillna(0,inplace=True)

直通车花费： 73451.31


### 第八步 淘宝客数据

In [16]:
index_ID = pd.read_excel(index_IDAddress)
index_ID = index_ID.replace(" ","",regex=True)

#新建列表，存放文件名（可以忽略，但是为了做的过程能心里有数，先放上）
fileName = []
#新建列表，存放每个文件数据框（每一个excel读取后存放在数据框）
frames = []
for root, dirs, files in os.walk(tbkFilesAddress):
    for file in files:
        #print(os.path.join(root,file))
        fileName.append(file)
#打印文件名
# print(fileName)



data_all=pd.DataFrame()

for i in range(len(fileName)):  
#     time.sleep(0.2)
    for j in range(len(index_ID.下载表名)):      
        if fileName[i].replace(" ","") ==index_ID.下载表名[j]:
            data=pd.read_excel(tbkFilesAddress + fileName[i])
            if data.empty:
                continue
            else:
                x = index_ID[index_ID['下载表名']==fileName[i].replace(" ","")]

                x.index=range(0,len(index_ID[index_ID['下载表名']==fileName[i].replace(" ","")]))
                data['ID']=json.loads(x['增加字段'][0])['宝贝id']
                data_all=data_all.append(data)
        else:
            continue
            
tbkData = data_all[['数据日期','预估付款佣金','预估付款服务费','ID']]
tbkData.insert(tbkData.shape[1],'淘宝客预估花费',tbkData['预估付款佣金']+tbkData['预估付款服务费'])
tbkData['数据日期'] = pd.to_datetime(tbkData['数据日期'])
tbkData['数据日期'] = tbkData['数据日期'].dt.strftime('%Y%m%d')
tbkData.insert(0,'日期商品ID',tbkData['数据日期']+tbkData['ID'])
tbkData = tbkData[['日期商品ID','淘宝客预估花费']]

tbkData.fillna(0,inplace=True)
tbkData_TSB = tbkData.groupby(['日期商品ID']).sum()
main_TSB_wx_GMV_ztc_tbk = pd.merge(main_TSB_wx_GMV_ztc,tbkData_TSB,left_index=True,right_index=True,how='left')


main_TSB_wx_GMV_ztc_tbk.fillna(0,inplace=True)

c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 第九步，超级推荐数据

In [17]:
cjtjData =  pd.read_excel(cjtjFileAddress)
cjtjData = cjtjData[['日期','计划','消耗']]
cjtjData['日期'] = pd.to_datetime(cjtjData['日期'])
cjtjData['日期'] = cjtjData['日期'].dt.strftime('%Y%m%d')
cjtjData.insert(0,'日期商品ID', cjtjData['日期'] + cjtjData['计划'].str[-12:])
cjtjData = cjtjData[['日期商品ID','消耗']]

cjtjData.fillna(0,inplace=True)

cjtjData_TSB = cjtjData.groupby(['日期商品ID']).sum()
print(type(cjtjData_TSB['消耗'][0]))
main_TSB_wx_GMV_ztc_tbk_cjtj = pd.merge(main_TSB_wx_GMV_ztc_tbk,cjtjData_TSB,left_index=True,right_index=True,how = 'left')

main_TSB_wx_GMV_ztc_tbk_cjtj.fillna(0,inplace=True)

<class 'numpy.float64'>


### 第十步，运费数据


In [18]:
yfData = pd.read_excel(yfFileAddress)
yfData['商品ID'] = yfData['商品ID'].apply(str)

main_TSB_wx_GMV_ztc_tbk_cjtj_yf = main_TSB_wx_GMV_ztc_tbk_cjtj
main_TSB_wx_GMV_ztc_tbk_cjtj_yf.insert(main_TSB_wx_GMV_ztc_tbk_cjtj_yf.shape[1],'运费单价',0)

main_TSB_wx_GMV_ztc_tbk_cjtj_yf.insert(0,'商品ID',main_TSB.index.str[8:])
main_TSB_wx_GMV_ztc_tbk_cjtj_yf.insert(0,'日期',main_TSB.index.str[0:8])

for i in range(len(main_TSB_wx_GMV_ztc_tbk_cjtj_yf)):
    for j in range(len(yfData)):
        if main_TSB_wx_GMV_ztc_tbk_cjtj_yf['商品ID'].iloc[i] == yfData['商品ID'].iloc[j]:
            main_TSB_wx_GMV_ztc_tbk_cjtj_yf['运费单价'].iloc[i] = yfData['运费单价'].iloc[j]

            
main_TSB_wx_GMV_ztc_tbk_cjtj_yf.insert(main_TSB_wx_GMV_ztc_tbk_cjtj_yf.shape[1],'运费金额',main_TSB_wx_GMV_ztc_tbk_cjtj_yf['AB单标志']*7+main_TSB_wx_GMV_ztc_tbk_cjtj_yf['运费单价']*(main_TSB_wx_GMV_ztc_tbk_cjtj_yf['宝贝数量']-main_TSB_wx_GMV_ztc_tbk_cjtj_yf['AB单标志']))

main_TSB_wx_GMV_ztc_tbk_cjtj_yf.fillna(0,inplace=True)

c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### 总表数据格式化输出

In [19]:
main_all = main_TSB_wx_GMV_ztc_tbk_cjtj_yf

In [20]:
main_all

,日期,商品ID,子订单金额,成交单价,宝贝数量,子订单金额（折算:邮费、优惠等）,其它价格1,是否AB单,备注金额,运营金额,退款总金额,AB单标志,支付金额,花费(分),淘宝客预估花费,消耗,运费单价,运费金额
日期商品ID,,,,,,,,,,,,,,,,,,
20210701539143464966,20210701,539143464966,336.00,168.000000,2,312.28,84.050000,0,0.0,168.1,0.00,0.0,312.28,0.00,0.00,0.00,21.0,42.0
20210701552028723427,20210701,552028723427,28.00,28.000000,1,28.00,10.400000,0,0.0,10.4,0.00,0.0,28.00,0.00,0.00,0.00,7.0,7.0
20210701564879845606,20210701,564879845606,18605.00,265.536232,69,18505.00,114.678261,8,60.0,7912.8,1626.80,7.0,21609.28,1467.66,232.48,41.55,37.8,2392.6
20210701567738064768,20210701,567738064768,18291.80,157.687931,116,15614.71,68.962069,11,636.5,7999.6,3266.05,17.0,17080.51,759.48,295.87,0.00,8.4,950.6
20210701569762951856,20210701,569762951856,558.20,69.775000,8,563.30,37.500000,0,16.0,300.0,0.00,0.0,612.99,0.00,2.10,0.00,7.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210708644818187878,20210708,644818187878,1012.00,253.000000,4,866.57,131.925000,0,0.0,527.7,0.00,0.0,0.00,24.18,0.00,0.00,14.0,56.0
20210708644911208119,20210708,644911208119,17.81,17.810000,1,17.81,7.300000,0,0.0,7.3,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0
20210708646119674804,20210708,646119674804,3286.10,298.736364,11,3286.10,180.818182,0,0.0,1989.0,0.00,0.0,0.00,148.42,0.00,0.00,0.0,0.0


In [21]:
main_all.insert(main_all.shape[1],'销售额(去退去差)',main_all['子订单金额（折算:邮费、优惠等）']-main_all['退款总金额']-main_all['备注金额'])
main_all.insert(main_all.shape[1],'毛利',(main_all['销售额(去退去差)']-main_all['运营金额']))
main_all.insert(main_all.shape[1],'平台费',main_all['子订单金额（折算:邮费、优惠等）']*0.085)
main_all.insert(main_all.shape[1],'净利',main_all['毛利']-main_all['花费(分)']-main_all['淘宝客预估花费']-main_all['消耗']-main_all['平台费']-main_all['运费金额'])

id_title = syjData_costAB[['日期商品ID','宝贝标题']]
id_title.drop_duplicates(subset='日期商品ID', keep='first', inplace=True)
id_title.reset_index(drop=True, inplace=True)

main_all_title = pd.merge(main_all, id_title, left_index = True,right_on='日期商品ID')
main_all_title.reset_index(drop=True, inplace=True)


c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### 十一、 AI投放

In [22]:
aitfData = pd.read_excel(aitfFileAddress)
aitfData_new = aitfData[['日期','消耗']]
aitfData_new.columns=['日期','AI投放']
aitfData_TSB = aitfData_new.groupby(['日期']).sum()
aitfData_TSB.index=pd.to_datetime(aitfData_TSB.index)

main_all_dateTSB = main_all[['日期','宝贝数量','是否AB单','支付金额','子订单金额','子订单金额（折算:邮费、优惠等）','备注金额','退款总金额','销售额(去退去差)','运营金额','毛利','淘宝客预估花费','花费(分)','消耗','运费金额','平台费','净利']].groupby(['日期']).sum()
main_all_dateTSB.index = pd.to_datetime(main_all_dateTSB.index)
main_all_dateTSB_aitf = pd.merge(main_all_dateTSB,aitfData_TSB,left_index=True,right_index=True)

main_all_per = main_all_dateTSB_aitf[['宝贝数量','是否AB单','支付金额','子订单金额','子订单金额（折算:邮费、优惠等）','备注金额','退款总金额','销售额(去退去差)','运营金额','毛利','淘宝客预估花费','花费(分)','消耗','AI投放','运费金额','平台费','净利']]

main_all_per['净利'] = main_all_per['净利'] - main_all_per['AI投放']

In [23]:
main_all_per

,宝贝数量,是否AB单,支付金额,子订单金额,子订单金额（折算:邮费、优惠等）,备注金额,退款总金额,销售额(去退去差),运营金额,毛利,淘宝客预估花费,花费(分),消耗,AI投放,运费金额,平台费,净利
日期,,,,,,,,,,,,,,,,,
2021-07-01,809,74,161624.02,154547.87,146217.01,2137.30,21995.92,122083.79,80378.2,41705.59,1308.31,9902.05,1150.93,1116.23758,11731.5,12428.44585,4068.11657
2021-07-02,751,96,153057.32,149719.15,142631.37,3190.38,27375.99,112065.00,75338.0,36727.00,1973.36,8955.24,1145.26,1299.85720,11611.5,12123.66645,-381.88365
2021-07-03,578,61,130626.93,123973.92,120900.97,2282.95,27643.00,90975.02,60893.1,30081.92,1159.36,9185.74,1123.94,1191.45202,8179.4,10276.58245,-1034.55447
2021-07-04,690,106,142305.64,138473.05,134305.00,2182.90,30077.45,102044.65,61887.8,40156.85,1208.01,10563.35,1259.33,1075.05844,10106.4,11415.92500,4528.77656
2021-07-05,722,99,148756.67,144326.45,139961.32,1900.43,34720.22,103340.67,65028.2,38312.47,1132.97,9679.11,1128.35,1062.17454,9847.6,11896.71220,3565.55326
2021-07-06,812,98,159710.43,156875.90,150143.13,2790.35,37057.70,110295.08,73105.9,37189.18,1084.02,9001.15,1120.20,988.22138,10258.7,12762.16605,1974.72257
2021-07-07,884,109,180726.05,178575.80,171532.10,1990.12,43178.35,126363.63,81016.0,45347.63,2497.82,7946.35,1124.59,940.89185,12085.1,14580.22850,6172.64965


In [24]:

# # 毛利率
main_all_per.insert(main_all_per.shape[1],'毛利率',main_all_per['毛利']/main_all_per['销售额(去退去差)'])

# 净利率
main_all_per.insert(main_all_per.shape[1],'净利率',main_all_per['净利']/main_all_per['销售额(去退去差)'])

# 成本占比
main_all_per.insert(main_all_per.shape[1],'成本占比',main_all_per['运营金额']/main_all_per['销售额(去退去差)'])

# 推广占比/销售额
main_all_per.insert(main_all_per.shape[1],'推广占比/销售额',(main_all_per['淘宝客预估花费']+main_all_per['花费(分)']+main_all_per['消耗']+main_all_per['AI投放'] )/ main_all_per['销售额(去退去差)'])

# 推广占比/GMV
main_all_per.insert(main_all_per.shape[1],'推广占比/GMV',(main_all_per['淘宝客预估花费']+main_all_per['花费(分)']+main_all_per['消耗']+main_all_per['AI投放'] )/ main_all_per['支付金额'])

# 退差占比
main_all_per.insert(main_all_per.shape[1],'退差占比',(main_all_per['退款总金额']+main_all_per['备注金额'])/main_all_per['销售额(去退去差)'])

# 平台占比
main_all_per.insert(main_all_per.shape[1],'平台占比',main_all_per['平台费']/main_all_per['子订单金额（折算:邮费、优惠等）'])

# 运费占比
main_all_per.insert(main_all_per.shape[1],'运费占比',main_all_per['运费金额']/main_all_per['销售额(去退去差)'])

# 退前折扣
main_all_per.insert(main_all_per.shape[1],'退前折扣',main_all_per['子订单金额（折算:邮费、优惠等）']/main_all_per['子订单金额'])

# 退后折扣
main_all_per.insert(main_all_per.shape[1],'退后折扣',main_all_per['销售额(去退去差)']/main_all_per['子订单金额'])

# 换成日期格式
main_all_per.replace([np.inf, -np.inf], np.nan, inplace=True)


In [25]:
main_all_new = main_all_title[['日期','商品ID','宝贝标题','宝贝数量','是否AB单','支付金额','子订单金额','子订单金额（折算:邮费、优惠等）','备注金额','退款总金额','销售额(去退去差)','运营金额','毛利','淘宝客预估花费','花费(分)','消耗','运费金额','平台费','净利']]

# # 毛利率
main_all_new.insert(main_all_new.shape[1],'毛利率',main_all_new['毛利']/main_all_new['销售额(去退去差)'])

# 净利率
main_all_new.insert(main_all_new.shape[1],'净利率',main_all_new['净利']/main_all_new['销售额(去退去差)'])

# 成本占比
main_all_new.insert(main_all_new.shape[1],'成本占比',main_all_new['运营金额']/main_all_new['销售额(去退去差)'])

# 推广占比/销售额
main_all_new.insert(main_all_new.shape[1],'推广占比/销售额',(main_all_new['淘宝客预估花费']+main_all_new['花费(分)']+main_all_new['消耗'] )/ main_all_new['销售额(去退去差)'])

# 推广占比/GMV
main_all_new.insert(main_all_new.shape[1],'推广占比/GMV',(main_all_new['淘宝客预估花费']+main_all_new['花费(分)']+main_all_new['消耗'] )/ main_all_new['支付金额'])

# 退差占比
main_all_new.insert(main_all_new.shape[1],'退差占比',(main_all_new['退款总金额']+main_all_new['备注金额'])/main_all_new['销售额(去退去差)'])

# 平台占比
main_all_new.insert(main_all_new.shape[1],'平台占比',main_all_new['平台费']/main_all_new['子订单金额（折算:邮费、优惠等）'])

# 运费占比
main_all_new.insert(main_all_new.shape[1],'运费占比',main_all_new['运费金额']/main_all_new['销售额(去退去差)'])

# 退前折扣
main_all_new.insert(main_all_new.shape[1],'退前折扣',main_all_new['子订单金额（折算:邮费、优惠等）']/main_all_new['子订单金额'])

# 退后折扣
main_all_new.insert(main_all_new.shape[1],'退后折扣',main_all_new['销售额(去退去差)']/main_all_new['子订单金额'])

main_all_new['日期'] = pd.to_datetime(main_all_new['日期'])
main_all_new['商品ID'] = pd.to_numeric(main_all_new['商品ID'])

#替换正负inf为NA，加inplace参数 
main_all_new.replace([np.inf, -np.inf], np.nan, inplace=True)


c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\zlych\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

In [26]:
main_all_per.columns=['销量(去正常退)','AB单数量','GMV','应付金额','销售额(去正常退)','客服登记退差','微信退差(AB单和微信销售)','销售额(去退去差)','运营成本','毛利','淘宝客','直通车','超级推荐','AI投放','快递费','平台费','净利润','毛利率','净利率','成本占比','推广占比/销售额', '推广占比/GMV', '退差占比',
       '平台占比', '运费占比', '退前折扣', '退后折扣']


main_all_days = main_all_per[['销量(去正常退)','AB单数量','GMV','应付金额','销售额(去正常退)','客服登记退差','微信退差(AB单和微信销售)','销售额(去退去差)','运营成本','毛利','毛利率','淘宝客','直通车','超级推荐','AI投放','快递费','平台费','净利润','净利率','成本占比','推广占比/销售额', '推广占比/GMV', '退差占比',
       '平台占比', '运费占比', '退前折扣', '退后折扣']]



main_all_new.columns = ['日期', '商品ID', '宝贝标题', '销量(去退)', 'AB单数量', 'GMV', '应付金额',
       '销售额（去退）', '客服退差', '微信退差(AB单和微信销售)', '销售额(去退去差)', '运营金额', '毛利',
       '淘宝客', '直通车', '超级推荐', '快递费', '平台费', '净利', '毛利率', '净利率', '成本占比',
       '推广占比/销售额', '推广占比/GMV', '退差占比', '平台占比', '运费占比', '退前折扣', '退后折扣']

main_all_new_ = main_all_new[['日期', '商品ID', '宝贝标题', '销量(去退)', 'AB单数量', 'GMV', '应付金额',
       '销售额（去退）', '客服退差', '微信退差(AB单和微信销售)', '销售额(去退去差)', '运营金额', '毛利','毛利率',
       '淘宝客', '直通车', '超级推荐', '快递费', '平台费', '净利','净利率', '成本占比',
       '推广占比/销售额', '推广占比/GMV', '退差占比', '平台占比', '运费占比', '退前折扣', '退后折扣']]



main_all_new_sort = main_all_new_.sort_values(by=['日期','GMV'],ascending=[True,False],inplace=False)

In [27]:
main_all_days.to_excel(dir + 'result\\旗舰店日度利润表7月{}.xlsx'.format(datetime.now().strftime('%Y-%m-%d %H_%M_%S')),sheet_name='旗舰店日度利润表')
main_all_per

main_all_new_sort.to_excel(dir + 'result\\单品日度利润表7月{}.xlsx'.format(datetime.now().strftime('%Y-%m-%d %H_%M_%S')),index=False,sheet_name='单品日度利润表')
# main_all_new